# Customer Analytics & A/B Testing in Python

## Chapter 1: Key Performance Indicators: Measuring Business Success

### Course Introduction and Overview:

#### What is A/B Testing?
* Test two or more different ideas against each other
* See which one empirically performs better

#### What is A/B Testing Important?
* No guessing
* Accurate Answers
* Rapidly iterate on ideas
* Establish causal relationships

#### Key performance indicators (KPIs)
* A/B tests are run to improve KPIs
* KPIs - metrics important to the organization
    * likelihood of a side-effect
    * revenue
    * conversion rate

#### Uncovering KPIs
* Experience
* Domain Knowledge
* Exploratory Data Analysis

### Identifying and understanding KPIs:

#### Example: Mobile App the offers mediation services
* Services:
    * Paid subscription
    * One-off in-app purchases
* Goals/KPIs:
    * Maintain high conversion rate
    
#### Dataset 1: User demographics

In [1]:
import pandas as pd

# load customer_demographics
customer_demographics = pd.read_csv("datasets/Customer dataset.csv")
customer_demographics['reg_date'] = pd.to_datetime(customer_demographics['reg_date']).dt.date

# print the head of customer_demographics
customer_demographics.head()

,uid,reg_date,device,gender,country,age
0,54030035.0,2017-06-29,and,M,USA,19
1,72574201.0,2018-03-05,iOS,F,TUR,22
2,64187558.0,2016-02-07,iOS,M,USA,16
3,92513925.0,2017-05-25,and,M,BRA,41
4,99231338.0,2017-03-26,iOS,M,FRA,59


#### Dataset 2: User actions

In [3]:
# load customer_subscriptions
app_purchases = pd.read_csv("datasets/In-App Purchases dataset.csv")

# pring the head of customer_subscriptions
app_purchases.head()

,date,uid,sku,price
0,2017-07-10,41195147,sku_three_499,499
1,2017-07-15,41195147,sku_three_499,499
2,2017-11-12,41195147,sku_four_599,599
3,2017-09-26,91591874,sku_two_299,299
4,2017-12-01,91591874,sku_four_599,599


#### KPI: conversion rate
* Choosing a KPI
    * Stable, generalizable KPIs are better than custom KPIs
    * Correlation with business factors
    
#### Match demographic to subscription data

In [4]:
purchase_data = customer_demographics.merge(app_purchases, how = 'inner', on = ['uid'])
purchase_data.head()

,uid,reg_date,device,gender,country,age,date,sku,price
0,92513925.0,2017-05-25,and,M,BRA,41,2017-10-20,sku_three_499,499
1,92513925.0,2017-05-25,and,M,BRA,41,2017-05-29,sku_two_299,299
2,92513925.0,2017-05-25,and,M,BRA,41,2017-08-23,sku_four_599,599
3,92513925.0,2017-05-25,and,M,BRA,41,2018-03-26,sku_six_1299,299
4,16377492.0,2016-10-16,and,M,BRA,20,2018-03-17,sku_one_199,199


### Exploratory analysis of KPIs:

#### Reminder: Conversion rate is just one KPI
* Most companies will have many KPIs
* Each serving a different purpose
* We are working through one of these cases: CVR

#### Group data: .groupby()

In [6]:
sub_data_grp = purchase_data.groupby(by = ['country', 'device'], axis = 0, as_index = False)
sub_data_grp

#### Aggregate data

In [7]:
sub_data_grp['price'].mean()

,country,device,price
0,BRA,and,412.985594
1,BRA,iOS,404.739300
2,CAN,and,406.826087
3,CAN,iOS,386.573964
4,DEU,and,402.474903
5,DEU,iOS,417.639798
6,FRA,and,418.377163
7,FRA,iOS,382.921569
8,TUR,and,433.913793
9,TUR,iOS,390.176471


#### Aggregate data: .agg()

In [8]:
sub_data_grp.agg({"price":['mean', 'min', 'max'], 'age':['mean', 'min', 'max']})

country device       price                 age        
                         mean min  max       mean min max
0      BRA    and  412.985594  99  899  23.913565  15  62
1      BRA    iOS  404.739300  99  899  23.975681  15  66
2      CAN    and  406.826087  99  899  23.282609  15  49
3      CAN    iOS  386.573964  99  899  25.023669  15  57
4      DEU    and  402.474903  99  899  23.814672  15  66
5      DEU    iOS  417.639798  99  899  20.773300  15  42
6      FRA    and  418.377163  99  899  23.200692  15  55
7      FRA    iOS  382.921569  99  899  24.407843  15  59
8      TUR    and  433.913793  99  899  24.004310  15  56
9      TUR    iOS  390.176471  99  899  22.591176  15  51
10     USA    and  399.000000  99  899  24.564738  15  65
11     USA    iOS  410.684783  99  899  24.144022  15  75

In [9]:
# Calculate the mean purchase price 
purchase_price_mean = purchase_data['price'].agg('mean')

# Examine the output 
print(purchase_price_mean)

406.77259604707973


In [10]:
# Group the data 
grouped_purchase_data = purchase_data.groupby(by = ['device', 'gender'])

# Aggregate the data
purchase_summary = grouped_purchase_data.agg({'price': ['mean', 'median', 'std']})

# Examine the results
print(purchase_summary)

                    price                   
                     mean median         std
device gender                               
and    F       400.747504    299  179.984378
       M       416.237308    499  195.001520
iOS    F       404.435330    299  181.524952
       M       405.272401    299  196.843197


### Calculating KPIs - A Practical Example

#### Goal - Comparing our KPIs
* Goal: Examine KPI of user conversion rate after free trial ends

#### Conversion rate : check max lapse date

In [29]:
import numpy as np
from datetime import datetime, timedelta
current_date = pd.to_datetime('2018-03-17')

In [30]:
# "Lapse date" = Date that the trial ended
print(purchase_data['reg_date'].max())

2018-03-17 00:00:00


#### KPI calculation: restrict users by lapse date

In [31]:
# latest lapse date: a week before today
max_lapse_data = current_date - timedelta(days = 7)

# restrict to usese lapsed before max_lapse_date
conv_sub_data = purchase_data[purchase_data['reg_date'] < max_lapse_date]

# count the users
total_users_count = conv_sub_data['price'].count()
total_users_count

8978

#### KPI calculation: restrict subscription date

In [33]:
# latest subscription date: within 7 days of lapsing
max_sub_date = conv_sub_data['reg_date'] + timedelta(days = 7)

# filter the users with non-zero subscription price
total_subs = conv_sub_data[
    (conv_sub_data['price'] > 0) &
    (conv_sub_data['date'] < max_sub_date)
]

# count the users
total_subs_count = total_subs['price'].count()
total_subs_count

190

#### KPI calculation: find the conversion rate

In [34]:
conversion_rate = total_subs_count / total_users_count
conversion_rate

0.021162842503898417

#### How long does it take to gain insight into a metric?
* Monthly conversion rate
    * would neeed to wait a month from lapse date
    * impractical
    
#### Exploratory data analysis
* Can reveal relationships between metrics and key results
* Can be tied to business metrics in important ways

#### Why is conversion rate important?
* Could potentially serve as a warning of potential problems later on

In [12]:
from datetime import datetime, timedelta

current_date = pd.to_datetime('2018-03-17')

# Compute max_purchase_date
max_purchase_date = current_date - timedelta(days=28)

# Convert dates
purchase_data['reg_date'] = pd.to_datetime(purchase_data['reg_date'])
purchase_data['date'] = pd.to_datetime(purchase_data['date'])

# Filter to only include users who registered before our max date
purchase_data_filt = purchase_data[purchase_data['reg_date'] < max_purchase_date]

# # Filter to contain only purchases within the first 28 days of registration
purchase_data_filt = purchase_data_filt[(purchase_data_filt['date'] <= 
                        purchase_data_filt['reg_date'] + timedelta(days=28))]

# Output the mean price paid per purchase
print(purchase_data_filt['price'].mean())

414.4237288135593


In [15]:
import numpy as np

# Set the max registration date to be one month before today
max_reg_date = current_date - timedelta(days=28)

# Find the month 1 values
month1 = np.where((purchase_data['reg_date'] < max_reg_date) &
                 (purchase_data['date'] < purchase_data['reg_date'] + timedelta(days=28)),
                  purchase_data['price'], 
                  np.NaN)
                 
# Update the value in the DataFrame
purchase_data['month1'] = month1

# Group the data by gender and device 
purchase_data_upd = purchase_data.groupby(by=['gender', 'device'], as_index=False) 

# Aggregate the month1 and price data 
purchase_summary = purchase_data_upd.agg(
                        {'price': ['mean', 'median'],
                        'month1': ['mean', 'median']})

# Examine the results 
print(purchase_summary)

  gender device       price             month1       
                       mean median        mean median
0      F    and  400.747504    299  388.204545  299.0
1      F    iOS  404.435330    299  432.587786  499.0
2      M    and  416.237308    499  413.705882  399.0
3      M    iOS  405.272401    299  433.313725  499.0


## Chapter 2: Exploring and Visualizing Customer Behavior

### Working with time series data in pandas:

#### Week Two Conversion Rate
* The rate at which people who have yet to subscribe, subscribe in their 2nd week, post lapse

#### Using the Timedelta Class

In [21]:
current_date = pd.to_datetime('2018-03-17')
max_lapse_date = current_date - timedelta(14)
conv_sub_data = purchase_data[purchase_data['reg_date'] < max_lapse_date]

#### Date Differences

In [22]:
sub_time = (conv_sub_data['date'] - conv_sub_data['reg_date'])
conv_sub_data['sub_time'] = sub_time.dt.days

/Users/MikaelaKlein/.venvs/venv_python_toolkit/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


#### Conversion Rate Calculation

In [23]:
conv_base = conv_sub_data[(conv_sub_data['sub_time'].notnull())|(conv_sub_data['sub_time'] > 7)]
total_users = len(conv_base)
total_users

8978

In [25]:
total_subs = np.where(conv_sub_data['sub_time'].notnull() & (conv_base['sub_time'] <= 14), 1, 0)
total_subs = sum(total_subs)
total_subs

357

In [26]:
conv_rate = total_subs/total_users
conv_rate

0.03976386723100914

### Creating time series graphs with matplotlib

#### Conversion Rate by Day

In [27]:
current_date = pd.to_datetime('2018-03-17')
max_lapse_date = current_date - timedelta(14)
conv_sub_data = purchase_data[purchase_data['reg_date'] < max_lapse_date]

sub_time = (conv_sub_data['date'] - conv_sub_data['reg_date'])
conv_sub_data['sub_time'] = sub_time

/Users/MikaelaKlein/.venvs/venv_python_toolkit/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
conversion_data = conv_sub_data.groupby(by = ['reg_date'], as_indes = False)
conversion_data = conversion_data.agg({"sub_time": [gcr7]})
conversion_data.columns = conversion_data.columns.droplevel(level=1)
conversion_data.head()

#### Plotting Daily Conversion Rate

In [ ]:
conv_data['lapse_date'] = pd.to_datetime(conv_data['lapse_date'])
conv_data.plot(x = 'lapse_date', y = 'sub_time')
plt.show()

#### Trends in Different Cohorts

In [ ]:
reformatted_cntry_data = pd.pivot_table(conv_data
                                        , values = ['sub_time']
                                        , columns = ['country']
                                        , index = ['reg_date']
                                        , fill_value = 0)
reformatted_cntry_data.columns.droplevel(level=[0])
reformatted_cntry_data.reset_index(inplace=True)
reformatted_cntry_data.plot(
    x = 'reg_date',
    y = ['BRA', 'FRA', 'DEU', 'TUR', 'USA', 'CAN'])
plt.show()

In [35]:
# Group the data and aggregate first_week_purchases
user_purchases = user_purchases.groupby(by=['reg_date', 'uid']).agg({'first_week_purchases': ['sum']})

# Reset the indexes
user_purchases.columns = user_purchases.columns.droplevel(level=1)
user_purchases.reset_index(inplace=True)

# Find the average amount purchased per user per day
user_purchases = user_purchases.groupby(by=['reg_date']).agg({'first_week_purchases': ['mean']})
user_purchases.columns = user_purchases.columns.droplevel(level=1)
user_purchases.reset_index(inplace=True)

# Plot the results
user_purchases.plot(x='reg_date', y='first_week_purchases')
plt.show()

NameError: name 'user_purchases' is not defined

### Understanding and Visualizing Trends

#### Further Techniques for Uncovering Trends
* Often, plotting a graph isn't enough
* Uncover 2 processing techniques

#### Subscribers Per Day

In [ ]:
usa_subscriptions = usa_subscriptions.groupby(by = ['sub_time'], as_index = False)
usa_subscriptions = usa_subscriptions.agg({'subs':['sum']})
usa_subscriptions.columns = usa_subscriptions.columns.droplevel(level = [1])
usa_subscriptions.plot(x = 'sub_date', y = 'subs')
plt.show()

#### Correcting for Seasonality
* Trailing Averages
    * Smoothing technique that setst the value for a given day as the average for the past n dyas
    * For weekly seasonality, let n = 7
    
#### Calculating Trailing Averages

In [ ]:
rolling_subs = usa_subscriptions['subs'].rolling(window = 7, center = False)
rolling_subs = rolling_subs.mean()
usa_subscriptions['rolling_subs'] = rolling_subs

#### Noisy Data
* Exponential Moving Average
    * Weights the points such that the earlier ones are weighted less than the more recent ones within our window
    * This pulls data back to any central trend, while maintaining any recent movement

#### Calculating an Exponential Moving Average

In [ ]:
exp_mean = high_sku_purchases['purchases'].exm(span = 30)
exp_mean = exp_mean.mean()
high_sku_purchases['exp_mean'] = exp_mean
high_sku_purchases.plot(x = 'date', y = 'exp_mean')

### Exploratory data analysis with time series data

#### Drop in New User Retention
* Limit view to closer range
* Split by Country & Device

#### Plotting Annotations

In [ ]:
releases['Date'] = pd.to_datetime(releases['Date'])
for row in releases.iterrows():
    if tmp['Event'] == 'iOS Release':
        plt.axvline(x = tmp['Date'], color = 'b', linestyle = '--')
    else:
        plt.axvline(x = tmp['Date'], color = 'r', linestyle = '--')
plt.show()

## Chapter 3: The Design and Application of A/B Testing

### Introduction to A/B Testing

#### Overview:
* An A/B test is an experiment in which you test 2 different values of the same variable against each other to determine which one is better.

### Initial A/B Test

#### Increasing Revenue through A/B Testing
* Generalizability of A/B Testing

#### A/B Testing Terminology:
* Response variable: Usually your KPI
* Factors & Variants: ex. red/blue paywall
* Experimental Unit: unit over which metrics are measured before aggregating the control & treatment group overall.
    * ex: a single user / user days
    

In [37]:
# Extract the 'day'; value from the timestamp
purchase_data['date'] = purchase_data['date'].dt.floor('d')

# Replace the NaN price values with 0 
purchase_data['price'] = np.where(np.isnan(purchase_data['price']), 0, purchase_data['price'])

# Aggregate the data by 'uid' & 'date'
purchase_data_agg = purchase_data.groupby(by=['uid', 'date'], as_index=False)
revenue_user_day = purchase_data_agg.sum()

# Calculate the final average
revenue_user_day = revenue_user_day.price.mean()
print(revenue_user_day)

407.95033407572384


### Preparing to run an A/B Test

#### Main Concerns in Designing a Test
1. Ensuring tests can be practically run.
2. Can derive meaningful results from it.

#### Test Sensitivity
* What % change would be meaningful to detect in our response variable? 1%? 20%?

#### Finding Revenue Per User

In [ ]:
purchase_data = demographics_data.merge(paywall_views, how = 'left', on = ['uid'])
purchase_data_agg = purchase_data.groupby(by = ['uid'], as_index = False)

total_revenue = purchase_data_agg['price'].sum()
total_revenue['price'] = np.where(np.isnan(total_revenue['price']), 0, total_revenue['price'])

avg_revenue = total_revenue['price'].mean()
avg_revenue

#### Evaluating Different Sensitivities
* Can evaluate by testing different view on the data w/ experience & intuition of your own.

In [ ]:
one_pct_lift = avg_revenue * 1.01
one_pct_lift

ten_pct_lift = avg_revenue * 1.1
ten_pct_lift

twenty_pct_lift = avg_revenue * 1.2
twenty_pct_lift

#### Standard Deviation

In [ ]:
revenue_variation = total_revenue.price.std()
revenue_variation

#### Variability of Revenue Per User

In [ ]:
revenue_variation/avg_revenue

#### Choosing our Experimental Unit & Response Variable
* Even though revenue is our ultimate goal, purchase conversion is a better response as it is more granular and more directly related to the change.

#### Standard Error for Conversion Rate

In [39]:
n = purchase_data['purchase'].count()
v = conv_rate * (1 - conv_rate)
var = v/n
se = var ** 0.5
print(var)
print(se)

KeyError: 'purchase'

In [43]:
ab_test_results = pd.read_csv('datasets/AB Testing Results.csv')
ab_test_results.head()

,uid,country,gender,spent,purchases,date,group,device
0,11115722,MEX,F,1595,5,2016-03-08,GRP B,I
1,11122053,USA,M,498,2,2017-07-14,GRP B,I
2,11128688,USA,F,2394,6,2017-09-17,GRP A,I
3,11130578,USA,F,1197,3,2017-11-30,GRP A,I
4,11130759,ESP,M,1297,3,2018-01-10,GRP B,A


In [51]:
ab_test_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45883 entries, 0 to 45882
Data columns (total 8 columns):
uid          45883 non-null int64
country      45883 non-null object
gender       45883 non-null object
spent        45883 non-null int64
purchases    45883 non-null int64
date         45883 non-null object
group        45883 non-null object
device       45883 non-null object
dtypes: int64(3), object(5)
memory usage: 2.8+ MB


In [49]:
purchase_data['uid'] = purchase_data['uid'].astype('int64')

In [54]:
purchase_data.merge(ab_test_results, how = 'outer', on = 'uid').head()#.shape

,uid,reg_date,device_x,gender_x,country_x,age,date_x,sku,price,month1,country_y,gender_y,spent,purchases,date_y,group,device_y
0,92513925,2017-05-25,and,M,BRA,41.0,2017-10-20,sku_three_499,499.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,92513925,2017-05-25,and,M,BRA,41.0,2017-05-29,sku_two_299,299.0,299.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,92513925,2017-05-25,and,M,BRA,41.0,2017-08-23,sku_four_599,599.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,92513925,2017-05-25,and,M,BRA,41.0,2018-03-26,sku_six_1299,299.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16377492,2016-10-16,and,M,BRA,20.0,2018-03-17,sku_one_199,199.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# Find the number of paywall views 
n = purchase_data.purchase.count()

# Calculate the quantitiy "v"
v = conv_rate * (1 - conv_rate) 

# Calculate the variance and standard error of the estimate
var = v / n 
se = var**0.5

print(var)
print(se)

AttributeError: 'DataFrame' object has no attribute 'purchase'

In [38]:
# Merge and group the datasets
purchase_data = demographics_data.merge(paywall_views,  how='inner', on=['uid'])
purchase_data.date = purchase_data.date.dt.floor('d')

# Group and aggregate our combined dataset 
daily_purchase_data = purchase_data.groupby(by=['date'], as_index=False)
daily_purchase_data = daily_purchase_data.agg({'purchase': ['sum', 'count']})

# Find the mean of each field and then multiply by 1000 to scale the result
daily_purchases = daily_purchase_data.purchase['sum'].mean()
daily_paywall_views = daily_purchase_data.purchase['count'].mean()
daily_purchases = daily_purchases * 1000
daily_paywall_views = daily_paywall_views * 1000

print(daily_purchases)
print(daily_paywall_views)

NameError: name 'demographics_data' is not defined

### Calculating Sample Sizes

#### Null Hypothesis
* Hypothesis that our control and treatment have the same impact on the response. Any observed difference is just due to randomness.
* If we can conclude that this is not the case, then we say our results are statistically significant and that there IS a difference.

#### Types of Error

H_0      | True          | False
---------|---------------|--------------
Accept   | Correct       | Type II Error
Reject   | Type I Error  | Correct

#### Confidence Interval
* Probability of NOT making a Type I Error
* Higher we make this value, larger sample needed
* Common value of this is 0.95

#### Statistical Power
* Probability of finding statistically significant results when the null hypothesis is FALSE.

#### Connecting the Different Componenets
* Power & Confidence Intervals are connected to Standard Error and sensitivity of our test.

#### Sample Size Function

In [83]:
from scipy import stats

In [84]:
def get_power(n, p1, p2, cl):
    alpha = 1 - cl
    qu = stats.norm.ppf(1 - alpha/2)
    diff = abs(p2 - p1)
    bp = (p1 + p2)/2
    v1 = p1 * (1 - p1)
    v2 = p2 * (1 - p2)
    bv = bp * (1 - bp)
    power_part_one = stats.norm.cdf((n**0.5 * diff - qu * (2*bv)**0.5)/(v1+v2)**0.5)
    power_part_two = 1 - stats.norm.cdf((n**0.5 * diff + qu * (2*bv)**0.5)/(v1+v2)**0.5)
    power = power_part_one + power_part_two
    return power

In [85]:
def get_sample_size(power, p1, p2, cl, max_n = 100000):
    n = 1
    while n <= max_n:
        tmp_power = get_power(n, p1, p2, cl)
        if tmp_power >= power:
            return n
        else:
            n = n + 1
    return "Increase Max N Value"

#### Calculating Our Needed Sample Size

In [71]:
sample_size_per_group = get_sample_size(0.8, conv_rate, conv_rate*1.1, 0.95)
sample_size_per_group

39720

In [73]:
p1 = 0.1
p2 = 0.12
cl = 0.95
n1 = 1000

# Look at the impact of sample size increase on power
n_param_one = get_power(n=1000, p1=p1, p2=p2, cl=cl)
n_param_two = get_power(n=2000, p1=p1, p2=p2, cl=cl)

# Look at the impact of confidence level increase on power
alpha_param_one = get_power(n=n1, p1=p1, p2=p2, cl=0.8)
alpha_param_two = get_power(n=n1, p1=p1, p2=p2, cl=0.95)
    
# Compare the ratios
print(n_param_two / n_param_one)
print(alpha_param_one / alpha_param_two)

1.7596440001351992
1.8857367092232278


## Chapter 4: Analyzing A/B Testing Results

### Analyzing the A/B test results

#### Confirming Test Results
* Confirm test was administered correctly
* Ensure that the data is sufficiently random

In [ ]:
test_results_grpd = test_results.groupby(by=['group'], as_index = False)
test_results_grpd.count()

* can also break out into country, gender, and device to ensure there is no bias

#### Statistical Significance:
* p-value: probability under the Null Hypothesis of obtaining a result as or more extreme than the one observed.
* represents a measure of the evidence against retaining the Null Hypothesis

### Understanding statistical significance

#### p-value Function

In [75]:
def get_pvalue(con_conv, test_conv, con_size, test_size):
    lift = -abs(test_conv - con_conv)
    scale_one = con_conv * (1 - con_conv) * (1/con_size)
    scale_two = test_conv * (1 - test_conv) * (1/test_size)
    scale_val = (scale_one + scale_two) ** 0.5
    p_value = 2 * stats.norm.cdf(lift, loc = 0, scale = scale_val)
    return p_value

In [77]:
con_conv = 0.034351
test_conv = 0.041984
con_size = 48236
test_size = 49867
p_value = get_pvalue(con_conv, test_conv, con_size, test_size)
p_value

4.257297485586909e-10

#### Finding the Power

In [78]:
get_power(test_size, con_conv, test_conv, 0.95)

0.9999925941372282

#### Confidence Intervals
* provides contextualization of the estimation process
* the conversion rate is a fixed quantity, the estimation is what's variable

#### Calculating Confidence Intervals

In [79]:
def get_ci(lift, alpha, sd):
    val = abs(stats.norm.ppf((1-alpha)/2))
    lwr_bnd = lift - val * sd
    upr_bnd = lift + val * sd
    return_val = (lwr_bnd, upr_bnd)
    return return_val

In [80]:
lift = test_conv - con_conv
sd = ((test_conv * (1 - test_conv))/test_size + (con_conv * (1 - con_conv))/con_size)**0.5
get_ci(lift, 0.95, sd)

(0.005237146294857827, 0.010028853705142175)

### Interpreting your test results

#### Communicating Your Test Results

Results            | Test Group          | Control Group
-------------------|---------------------|------------------
Sample Size        | 7030                | 6970
Run Time           | 2 Weeks             | 2 Weeks 
Mean               | 3.12                | 2.69
Variance           | 3.20                | 2.64

* Estimated Lift: 0.56*
* Confidence Interval: 0.56 +- 0.4
* significance at the 0.05 level

#### Plotting the Distribution

In [82]:
import matplotlib.pyplot as plt

mean_control = 0.090965
mean_test = 0.102005
var_control = (mean_control * (1 - mean_control))/58583
var_test = (mean_test * (1 - mean_test))/56350
control_line = np.linspace(-3 * var_control ** 0.5 + mean_control, 3 * var_control ** 0.5 + mean_control, 100)
test_line = np.linspace(-3 * var_test ** 0.5 + mean_test, 3 * var_test ** 0.5 + mean_test, 100)
plt.plot(control_line, mlab.normpdf(control_line, mean_control, var_control ** 0.5))
plt.show()

NameError: name 'mlab' is not defined

#### Plotting the Difference of Distributions

In [ ]:
lift = mean_test - mean_control
var = var_test + var_control
diff_line = np.linspace(-3 * var ** 0.5 + lift, 3 * var ** 0.5 + lift, 100)
plt.plot(diff_line, mlab.normpdf(diff_line, lift, var**0.5))
plt.show()

#### Plotting the Confidence Interval

In [ ]:
section = np.arange(0.007624, 0.01445, 1/10000)
plt.fill_between(section, mlab.normpdf(section, lift, var**0.5))
plt.plot(diff_line, mlab.normpdf(diffline, lift, var**0.5))
plt.show()